In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#install pygaggle before transformers (first create the index using pyserini then install pygaggle)
!pip install pygaggle==0.0.2
!pip install transformers==4.17.0
import transformers
import pygaggle

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.transformer import MonoT5, DuoT5
reranker =  MonoT5()

In [4]:
import pandas as pd
from tqdm import tqdm
from xml.dom import minidom

In [30]:
relevant_bm25=pd.read_csv("/content/drive/MyDrive/Touche/2020/relevant_bm25_2020.csv")
relevant_bm25.head()

,Unnamed: 0,title_id,title,doc_id,score,content
0,0,1,\nWhat is the difference between sex and love?\n,clueweb12-0818wb-26-13074,7.1866,toofly nyc » hip hop culture toofly nyc clien...
1,1,1,\nWhat is the difference between sex and love?\n,clueweb12-0010wb-33-27298,7.1624,relational quantum mechanics stanford encyclo...
2,2,1,\nWhat is the difference between sex and love?\n,clueweb12-0103wb-41-14265,6.9118,lecture series science software engineering...
3,3,1,\nWhat is the difference between sex and love?\n,clueweb12-1013wb-51-17839,6.4538,ethical buyers guide toy cars trains planes s...
4,4,1,\nWhat is the difference between sex and love?\n,clueweb12-1400tw-39-23968,6.2936,video video cbr tv movie trailersclips tv tra...


# Training different Rerankers

In [ ]:
# Mono T5 reranker
new=[]
for title in tqdm(relevant_bm25["title"].unique()):
  passages= relevant_bm25.loc[relevant_bm25["title"]==title]
  texts = [ Text(p[1].content, {'docid': p[1].doc_id}, 0) for p in passages.iterrows()]
  query = Query(title)
  reranked = reranker.rerank(query, texts)
  for i in range(0, 1000):
    new.append({"doc_id":reranked[i].metadata["docid"], "Score":reranked[i].score, "title":title})

In [ ]:
df=pd.DataFrame(new)

In [ ]:
df.to_csv("/content/drive/MyDrive/Touche/2020/reranked_2020_mono_t5.csv")

In [ ]:
#distilbert reranker
#Load the model
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/msmarco-distilbert-base-tas-b')



In [33]:
new=[]
for title in tqdm(relevant_bm25["title"].unique()):
  query=title
  docs= relevant_bm25.loc[relevant_bm25["title"]==title]["doc_id"].tolist()
  #Encode query and documents
  query_emb = model.encode(query)
  doc_emb = model.encode(docs)

  #Compute dot score between query and all document embeddings
  scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()

  #Combine docs & scores
  doc_score_pairs = list(zip(docs, scores))

  #Sort by decreasing score
  doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)

  #Output passages & scores
  for doc, score in tqdm(doc_score_pairs):
    new.append({"doc_id":doc, "Score":score, "title":title})

100%|██████████| 50/50 [01:14<00:00,  1.49s/it]


In [35]:
df=pd.DataFrame(new)
df.shape

(50000, 3)

In [36]:
df.to_csv("/content/drive/MyDrive/Touche/2020/reranked_2020_distilbert.csv")

# Evaluation

In [37]:
#defining a functions to get all topics
def parse_xml(path):
  answer_list = []
  xmldoc = minidom.parse(path)
  itemlist = xmldoc.getElementsByTagName('topics')
  topic_list = itemlist[0].getElementsByTagName('topic')
  for topic in topic_list:
    tuple_for_add = tuple((topic.getElementsByTagName('number')[0].firstChild.nodeValue, topic.getElementsByTagName('title')[0].firstChild.nodeValue))
    answer_list.append(tuple_for_add)
  parsed=pd.DataFrame(answer_list, columns=["Number","Title"])
  return parsed

In [38]:
topics_2020=parse_xml("/content/drive/MyDrive/Touche/topics-task-2-2020.xml")
topics_2020.head()

,Number,Title
0,1,\nWhat is the difference between sex and love?\n
1,2,"\nWhich is better, a laptop or a desktop?\n"
2,3,"\nWhich is better, Canon or Nikon?\n"
3,4,\nWhat are the best dish detergents?\n
4,5,\nWhat are the best cities to live in?\n


In [44]:
# getting the reranked documents (Mono T5)
solution=pd.read_csv("/content/drive/MyDrive/Touche/2020/reranked_2020__mono_t5.csv")
solution.head()

,Unnamed: 0,doc_id,Score,title
0,0,clueweb12-0818wb-26-13074,-13.594674,\nWhat is the difference between sex and love?\n
1,1,clueweb12-0010wb-33-27298,-13.818763,\nWhat is the difference between sex and love?\n
2,2,clueweb12-0103wb-41-14265,-13.821753,\nWhat is the difference between sex and love?\n
3,3,clueweb12-1013wb-51-17839,-13.743243,\nWhat is the difference between sex and love?\n
4,4,clueweb12-1400tw-39-23968,-14.184422,\nWhat is the difference between sex and love?\n


In [47]:
# getting the reranked documents (Distilbert)
solution=pd.read_csv("/content/drive/MyDrive/Touche/2020/reranked_2020_distilbert.csv")
solution.head()

,Unnamed: 0,doc_id,Score,title
0,0,clueweb12-1101wb-70-17195,81.122955,\nWhat is the difference between sex and love?\n
1,1,clueweb12-0303wb-75-12068,80.287552,\nWhat is the difference between sex and love?\n
2,2,clueweb12-0009wb-79-27512,80.253891,\nWhat is the difference between sex and love?\n
3,3,clueweb12-1006wb-25-23874,80.233803,\nWhat is the difference between sex and love?\n
4,4,clueweb12-1408wb-30-10865,80.119133,\nWhat is the difference between sex and love?\n


In [40]:
new_rel_2020=pd.read_csv("/content/drive/MyDrive/Touche/2020/new_rel_2020.csv")
new_rel_2020["qid"] = new_rel_2020["qid"].astype(str)
new_rel_2020.head()

,Unnamed: 0,qid,no,doc,rel
0,0,1,0,clueweb12-0001wb-05-12311,0
1,3,1,0,clueweb12-1217wb-47-14048,0
2,7,1,0,clueweb12-1214wb-88-29751,2
3,8,1,0,clueweb12-0307wb-05-31620,0
4,9,1,0,clueweb12-1500wb-68-15142,0


In [48]:
from collections import defaultdict
ground_truth_dict = defaultdict(list)
rel0_truth_dict = defaultdict(list)
rel1_truth_dict = defaultdict(list)
rel2_truth_dict = defaultdict(list)

for i_, d_, x_ in zip(new_rel_2020['qid'], new_rel_2020['doc'], new_rel_2020['rel']):
    i_ = str(i_)
    d_ = str(d_)    
    if int(x_) > 0:
        ground_truth_dict[i_].append(d_)
    if int(x_) == 0:
        rel0_truth_dict[i_].append(d_)
    if int(x_) == 1:
        rel1_truth_dict[i_].append(d_)
    if int(x_) == 2:
        rel2_truth_dict[i_].append(d_)

In [56]:
# Run loop for topics_2020 data, store data according to ranking
solution_dict_opt = {} # topic_id, corresponding_document list.
for title, number in tqdm(zip(topics_2020['Title'], topics_2020['Number'])):
  d_list=[]
  d_list=solution.loc[solution["title"]==title].sort_values(by="Score",ascending=False).iloc[0:1000,:]["doc_id"].tolist()
  solution_dict_opt[number] = d_list
solution_dict=solution_dict_opt

50it [00:00, 129.29it/s]


In [57]:
# the final dictionaries for basic metric evaluation and analysis.
# Average percentage common, Hit-once and Hit-all metric basic definition.
hit_one = 0
hit_all = 0
total = 50
per_comm_avg = 0

for id_i, doc_i in ground_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in solution_dict.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit_one += 1
            if doc_j.issuperset(doc_i):
                hit_all += 1
            per_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'Hit one: {round(hit_one / total, 4)}')
print(f'Hit all: {round(hit_all / total, 4)}')
print(f'Average common ratio: {round(per_comm_avg / total, 4)}')

hit0_one = 0
hit0_all = 0
per0_comm_avg = 0

for id_i, doc_i in rel0_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in solution_dict.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit0_one += 1
            if doc_j.issuperset(doc_i):
                hit0_all += 1
            per0_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'Zero Relevance, Hit one: {round(hit0_one / total, 4)}')
print(f'Zero Relevance, Hit all: {round(hit0_all / total, 4)}')
print(f'Zero Relevance, Average common ratio: {round(per0_comm_avg / total, 4)}')

hit1_one = 0
hit1_all = 0
per1_comm_avg = 0

for id_i, doc_i in rel1_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in solution_dict.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit1_one += 1
            if doc_j.issuperset(doc_i):
                hit1_all += 1
            per1_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'One Relevance, Hit one: {round(hit1_one / total, 4)}')
print(f'One Relevance, Hit all: {round(hit1_all / total, 4)}')
print(f'One Relevance, Average common ratio: {round(per1_comm_avg / total, 4)}')

hit2_one = 0
hit2_all = 0
per2_comm_avg = 0

for id_i, doc_i in rel2_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in solution_dict.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit2_one += 1
            if doc_j.issuperset(doc_i):
                hit2_all += 1
            per2_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'Two Relevance, Hit one: {round(hit2_one / total, 4)}')
print(f'Two Relevance, Hit all: {round(hit2_all / total, 4)}')
print(f'Two Relevance, Average common ratio: {round(per2_comm_avg / total, 4)}')

Hit one: 1.0
Hit all: 0.6
Average common ratio: 0.9456
Zero Relevance, Hit one: 1.0
Zero Relevance, Hit all: 0.24
Zero Relevance, Average common ratio: 0.8499
One Relevance, Hit one: 1.0
One Relevance, Hit all: 0.72
One Relevance, Average common ratio: 0.9455
Two Relevance, Hit one: 0.88
Two Relevance, Hit all: 0.72
Two Relevance, Average common ratio: 0.8541


In [28]:
# Mono T5

#All 1000 rereanked documents
# Hit one: 1.0
# Hit all: 0.6
# Average common ratio: 0.9456
# Zero Relevance, Hit one: 1.0
# Zero Relevance, Hit all: 0.24
# Zero Relevance, Average common ratio: 0.8499
# One Relevance, Hit one: 1.0
# One Relevance, Hit all: 0.72
# One Relevance, Average common ratio: 0.9455
# Two Relevance, Hit one: 0.88
# Two Relevance, Hit all: 0.72
# Two Relevance, Average common ratio: 0.8541


##############################################
#Top  500 rereanked documents

# Hit one: 1.0
# Hit all: 0.5
# Average common ratio: 0.9277
# Zero Relevance, Hit one: 1.0
# Zero Relevance, Hit all: 0.1
# Zero Relevance, Average common ratio: 0.7187
# One Relevance, Hit one: 1.0
# One Relevance, Hit all: 0.6
# One Relevance, Average common ratio: 0.9249
# Two Relevance, Hit one: 0.88
# Two Relevance, Hit all: 0.68
# Two Relevance, Average common ratio: 0.8434

############################################
#Top  100 rereanked documents

# Hit one: 1.0
# Hit all: 0.06
# Average common ratio: 0.6474
# Zero Relevance, Hit one: 1.0
# Zero Relevance, Hit all: 0.0
# Zero Relevance, Average common ratio: 0.2503
# One Relevance, Hit one: 0.92
# One Relevance, Hit all: 0.1
# One Relevance, Average common ratio: 0.5362
# Two Relevance, Hit one: 0.86
# Two Relevance, Hit all: 0.3
# Two Relevance, Average common ratio: 0.6815

############################################


In [ ]:
# Mono T5

#All 1000 rereanked documents
# Hit one: 1.0
# Hit all: 0.6
# Average common ratio: 0.9456
# Zero Relevance, Hit one: 1.0
# Zero Relevance, Hit all: 0.24
# Zero Relevance, Average common ratio: 0.8499
# One Relevance, Hit one: 1.0
# One Relevance, Hit all: 0.72
# One Relevance, Average common ratio: 0.9455
# Two Relevance, Hit one: 0.88
# Two Relevance, Hit all: 0.72
# Two Relevance, Average common ratio: 0.8541


##############################################
#Top  500 rereanked documents

# Hit one: 1.0
# Hit all: 0.0
# Average common ratio: 0.4552
# Zero Relevance, Hit one: 0.96
# Zero Relevance, Hit all: 0.0
# Zero Relevance, Average common ratio: 0.4388
# One Relevance, Hit one: 0.96
# One Relevance, Hit all: 0.06
# One Relevance, Average common ratio: 0.4587
# Two Relevance, Hit one: 0.84
# Two Relevance, Hit all: 0.06
# Two Relevance, Average common ratio: 0.4418

############################################
#Top  100 rereanked documents

# Hit one: 0.74
# Hit all: 0.0
# Average common ratio: 0.0899
# Zero Relevance, Hit one: 0.74
# Zero Relevance, Hit all: 0.0
# Zero Relevance, Average common ratio: 0.0764
# One Relevance, Hit one: 0.46
# One Relevance, Hit all: 0.0
# One Relevance, Average common ratio: 0.0804
# Two Relevance, Hit one: 0.46
# Two Relevance, Hit all: 0.0
# Two Relevance, Average common ratio: 0.0876

############################################


#Duo T5

In [ ]:
from pygaggle.rerank.transformer import DuoT5

In [ ]:
reranker =  DuoT5()

In [ ]:
mono=pd.read_csv("/content/drive/MyDrive/Touche/reranked.csv")
mono.head()

In [ ]:
new=[]
for title in tqdm(mono["title"].unique()):
  x= mono.loc[mono["title"]==title].sort_values(by="Score",ascending=False).iloc[0:100,:]
  passages=x.join(relevant_bm25[["doc_id","content"]], how="inner",lsuffix='_caller', rsuffix='_other')
  texts = [ Text(p[1].content, {'docid': p[1].doc_id_caller}, p[1].Score) for p in passages.iterrows()]
  query = Query(title)
  reranked = reranker.rerank(query, texts)
  for i in range(0, 100):
    new.append({"doc_id":reranked[i].metadata["docid"], "Score":reranked[i].score, "title":title})